# 라이브러리

In [1]:
import librosa
import pandas as pd
import numpy as np
import random
import shutil
import os

# Meta File

In [2]:
df_path = "/home/woongjae/noise-tracing/Datasets/2-DSD-corpus/all_meta_fake.csv"

meta_df = pd.read_csv(df_path, sep=",")

meta_df.head()

,utt,Utterence name (file name),Speaker name,Model,Language,Gender,Age,Source link,Noise type 1,label,group,TTS or VC,Is multi-speaker?,path,subset
0,TTS_Sasha_12,TTS_Sasha_12.wav,Sasha,-,Swedish,Female,Teenager,https://elevenlabs.io/speech-synthesis,-,spoof,Elevenlabs,TTS,No,Synthesizers/Elevenlabs/TTS_Sasha_12.wav,train
1,TTS_Paul_99,TTS_Paul_99.wav,Paul,-,Korean,Male,Adult,https://elevenlabs.io/speech-synthesis,-,spoof,Elevenlabs,TTS,No,Synthesizers/Elevenlabs/TTS_Paul_99.wav,train
2,TTS_Patrick_41,TTS_Patrick_41.wav,Patrick,-,French,Male,Adult,https://elevenlabs.io/speech-synthesis,-,spoof,Elevenlabs,TTS,No,Synthesizers/Elevenlabs/TTS_Patrick_41.wav,train
3,TTS_Jeremy_49,TTS_Jeremy_49.wav,Jeremy,-,Hindi,Male,Adult,https://elevenlabs.io/speech-synthesis,-,spoof,Elevenlabs,TTS,No,Synthesizers/Elevenlabs/TTS_Jeremy_49.wav,train
4,TTS_Joseph_30,TTS_Joseph_30.wav,Joseph,-,Arabic,Male,Adult,https://elevenlabs.io/speech-synthesis,-,spoof,Elevenlabs,TTS,No,Synthesizers/Elevenlabs/TTS_Joseph_30.wav,train


In [5]:
print(meta_df['group'].unique())
print(meta_df['group'].value_counts())

['Elevenlabs' 'DDDM-VC' 'VITS-TTS' 'SeamlessM4T-TTS' 'MMSTTS' 'QuickVC'
 'VITS-AIHUB' 'Diff-HierVC' 'Tortoise-TTS' 'StarGANv2-VC' 'OpenVoice'
 'MeloTTS' 'StyleTTS2']
VITS-AIHUB         16300
Elevenlabs          5000
VITS-TTS            4100
Tortoise-TTS        2900
DDDM-VC             2000
QuickVC             2000
Diff-HierVC         2000
StarGANv2-VC        2000
OpenVoice           1300
StyleTTS2           1300
SeamlessM4T-TTS     1100
MeloTTS             1000
MMSTTS               900
Name: group, dtype: int64


In [16]:
# print(meta_df['Speaker name'].unique())
# print(meta_df['Speaker name'].value_counts())
print(meta_df[meta_df['Language']=='English'].value_counts())

utt                          Utterence name (file name)       Speaker name    Model      Language  Gender  Age    Source link                                         Noise type 1  label  group         TTS or VC  Is multi-speaker?  path                                                       subset
MMSTTS_eng_000001            MMSTTS_eng_000001.wav            MMSTTS_English  VITS       English   Male    Adult  https://huggingface.co/facebook/mms-tts             -             spoof  MMSTTS        TTS        No                 Synthesizers/MMSTTS/MMSTTS_eng_000001.wav                  dev       1
VITS_p284_to_p285_5          VITS_p284_to_p285_5.wav          p285            VITS       English   Male    Adult  https://github.com/quickvc/QuickVC-VoiceConversion  -             spoof  QuickVC       VC         No                 Synthesizers/QuickVC/VITS_p284_to_p285_5.wav               train     1
VITS_p284_to_p285_25         VITS_p284_to_p285_25.wav         p285            VITS       English   

In [22]:
df = pd.read_csv(df_path)
df['Language'] = df['Language'].str.strip()
df_en = df[df['Language'] == 'English']

# 🔧 Speaker name 정제
df_en['Speaker name'] = df_en['Speaker name'].str.strip().str.lower()

# ✅ 다시 speaker당 샘플 수 확인
speaker_counts = df_en['Speaker name'].value_counts()
eligible_speakers = speaker_counts[speaker_counts >= 100]

print(f"✅ Cleaned English speaker 중 100개 이상인 speaker 수: {len(eligible_speakers)}")
print(f"📦 총 샘플 수: {eligible_speakers.sum()}")

✅ Cleaned English speaker 중 100개 이상인 speaker 수: 152
📦 총 샘플 수: 17300


/tmp/ipykernel_4038601/2076163805.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en['Speaker name'] = df_en['Speaker name'].str.strip().str.lower()


### 모든 그룹의 spoof 샘플 저장하기

In [4]:
meta_path = "/home/woongjae/noise-tracing/Datasets/2-DSD-corpus/all_meta_fake.csv"
output_dir = "/home/woongjae/noise-tracing/new_dataset/Dataset/Spoof/Clean"
base_dir = "/home/woongjae/noise-tracing/Datasets/2-DSD-corpus"
os.makedirs(output_dir, exist_ok=True)

# 메타데이터 불러오기
df = pd.read_csv(meta_path)

# ✅ Language == English만 필터링
df = df[df['Language'] == 'English']

# speaker별 샘플 개수 확인
speaker_counts = df['Speaker name'].value_counts()
eligible_speakers = speaker_counts[speaker_counts >= 100].index

# spoof 샘플 저장용 리스트
new_meta = []

# 필터링된 speaker 대상으로 100개씩 랜덤 추출 및 복사
for speaker in eligible_speakers:
    speaker_df = df[df['Speaker name'] == speaker].sample(n=100, random_state=42)
    
    for _, row in speaker_df.iterrows():
        rel_path = row['path']
        abs_path = os.path.join(base_dir, rel_path)
        dst = os.path.join(output_dir, os.path.basename(rel_path))

        # 파일 복사
        shutil.copy(abs_path, dst)

        # 메타데이터 항목 생성
        new_meta.append({
            'File_path': dst,
            'utt': row['utt'],
            'speaker ID': row['Speaker name'],
            'gender': row['Gender'],
            'Group': row['group'],
            'Label': 'spoof'
        })

# 메타데이터 저장
meta_out = "/home/woongjae/noise-tracing/new_dataset/meta_new_DSDCorpus.csv"
pd.DataFrame(new_meta).to_csv(meta_out, index=False)
print(f"✅ 메타데이터 저장 완료: {meta_out} (샘플 수: {len(new_meta)})")

✅ 메타데이터 저장 완료: /home/woongjae/noise-tracing/new_dataset/meta_new_DSDCorpus.csv (샘플 수: 15200)
